In [1]:
import warnings
warnings.filterwarnings("ignore")

import spacy
nlp = spacy.load('en_coref_sm')             # 引用小的英语指代模型

In [2]:
doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)                      # 是否有指代？
print(doc._.coref_clusters)                 # 共指链组成的列表
print(doc._.coref_resolved)                 # 将代词用指代簇中的主要描述代替后的文本

True
[My sister: [My sister, She], a dog: [a dog, him]]
My sister has a dog. My sister loves a dog.


下面的例子中我们可以获得更多详细信息。

In [3]:
doc = nlp(u'My sister has a dog. She loves him')
print(doc.text)                       # 重新获得文本内容,string对象
for clust in doc._.coref_clusters:
    clust_id = clust.i                # 该指代在本文档中所有指代中的序号 
    main_mention = clust.main         # 主描述，共指链中的代表词语
    print("appearance of cluster %d: %s" % (clust_id,main_mention))
    for mention in clust.mentions:
        print("\t%s at [%d,%d]" % (mention.text,mention.start_char,mention.end_char))

My sister has a dog. She loves him
appearance of cluster 0: My sister
	My sister at [0,9]
	She at [21,24]
appearance of cluster 1: a dog
	a dog at [14,19]
	him at [31,34]


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这个例子揭示了共指消解中的3层的结构，Doc[文档]-cluster[共指链，代表同一实体的所有对象]-mention[共指链中的描述，实体的一个描述]。其中我们得到了共指链的代表词和每个个别描述在原文中的位置，这些将有助于我们还原文本或干一些别的事情（尽管已经有了coref_resolved，有时候我们还是需要做出一些特别的修改，后面的一篇文章中我就会用到它们。）。

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;以上例子已经揭示了这个工具的主要功能。这里再提供一些细节，Document下面有3种结构，下面列出它们支持的方法。

In [4]:
doc = nlp(u'My sister has a dog. She loves him')

clust1 = doc._.coref_clusters[1]                        # Cluster，一个共指链
print(clust1.i)                                         # 该指代在本文档中所有指代中的序号 
print(clust1.mentions)                                  # 本共指链中的所有对象
print(clust1.main)                                      # 主描述，共指链中的代表词语
print(clust1.mentions[-1]._.coref_cluster)              # _.coref_cluster【该对象所处的共指链】

1
[a dog, him]
a dog
a dog: [a dog, him]


In [5]:
token = doc[-1]                                         # token，相当于一个词语
print(token._.in_coref)                                 # 是否在至少一个共指链中
print(token._.coref_clusters)                           # 包括这个token的所有共指链
# 遍历所有token
for tok in doc:
    print(tok,end = " ")
print()

True
[a dog: [a dog, him]]
My sister has a dog . She loves him 


In [6]:
span = doc[-1:]                                         # span，doc中的一个部分
print(span._.is_coref)                                  # 是否至少包含了一个共指对象
print(span._.coref_cluster.main)
print(span._.coref_cluster.main._.coref_cluster)

True
a dog
a dog: [a dog, him]
